In [13]:
import ee
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

import os
from os import path as osp
from tqdm.auto import tqdm
import json

import ee_utils # library of functions

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library. Replace with your relevant project title
ee.Initialize(project ='ee-georgethulsey')

random.seed(123)

### export ee data functions

In [2]:
def _get_all_feature_bands():
    """Returns list of all bands corresponding to features."""
    return (ee_utils.DATA_BANDS[ee_utils.DataType.ELEVATION_SRTM] +
            ['chili'] +
            ee_utils.DATA_BANDS[ee_utils.DataType.IMPERVIOUS] + 
            ['water'] + 
            ['population'] + 
            ['fuel1','fuel2','fuel3'] +

            ee_utils.DATA_BANDS[ee_utils.DataType.VEGETATION_VIIRS] +
            ee_utils.DATA_BANDS[ee_utils.DataType.DROUGHT_GRIDMET] +
            ee_utils.DATA_BANDS[ee_utils.DataType.WEATHER_GRIDMET] +
            ['modis_PrevFireMask']+['viirs_PrevFireMask']
            )

def _get_all_response_bands():
    """Returns list of all bands corresponding to labels."""
    return ['modis_FireMask'] + ['viirs_FireMask']


def _add_index(i, bands):
    """Appends the index number `i` at the end of each element of `bands`."""
    return [f'{band}_{i}' for band in bands]

In [3]:
def _get_all_image_collections(viirs_info = None):
    """Gets all the image collections and corresponding time sampling."""
    image_collections = {
      'drought':
          ee_utils.get_image_collection(ee_utils.DataType.DROUGHT_GRIDMET),
      'vegetation':
          ee_utils.get_image_collection(ee_utils.DataType.VEGETATION_VIIRS),
      'weather':
          ee_utils.get_image_collection(ee_utils.DataType.WEATHER_GRIDMET),
      'modis_fire':
          ee_utils.get_image_collection(ee_utils.DataType.FIRE_MODIS),
      'viirs_fire':
          ee_utils.get_image_collection(ee_utils.DataType.FIRE_VIIRS,viirs_info = viirs_info),
    }
    time_sampling = {
      'drought':
          ee_utils.DATA_TIME_SAMPLING[ee_utils.DataType.DROUGHT_GRIDMET],
      'vegetation':
          ee_utils.DATA_TIME_SAMPLING[ee_utils.DataType.VEGETATION_VIIRS],
      'weather':
          ee_utils.DATA_TIME_SAMPLING[ee_utils.DataType.WEATHER_GRIDMET],
      'modis_fire':
          ee_utils.DATA_TIME_SAMPLING[ee_utils.DataType.FIRE_MODIS],
      'viirs_fire':
          ee_utils.DATA_TIME_SAMPLING[ee_utils.DataType.FIRE_VIIRS],
    }
    return image_collections, time_sampling

def _verify_feature_collection(
    feature_collection
    ):
    """Verifies the feature collection is valid.
    
    If the feature collection is invalid, resets the feature collection.
    
    Args:
    feature_collection: An EE feature collection.
    
    Returns:
    `(feature_collection, size)` a tuple of the verified feature collection and
    its size.
    """
    try:
        size = int(feature_collection.size().getInfo())
        
        # print("Verified a feature collection of size",size) # debugging
        
    except ee.EEException:
        # print('EE exception thrown, resetting') # debugging
        
        # Reset the feature collection
        feature_collection = ee.FeatureCollection([])
        size = 0
    
    return feature_collection, size

In [11]:
def _get_time_slices(
    window_start,
    window = 1,
    projection = ee.Projection('EPSG:4326'),  # Defer calling until called by test code
    resampling_scale = 500, # units: meters
    lag = 0,
):
    """Extracts the time slice features.
    
    Args:
    window_start: Start of the time window over which to extract data.
    window: Length of the window (in days).
    projection: projection to reproject all data into.
    resampling_scale: length scale to resample data to.
    lag: Number of days before the fire to extract the features.
    
    Returns:
    A list of the extracted EE images.
    """
    projection = projection.atScale(resampling_scale) # reprojection flow is a bit confusing
    
    window_end = window_start.advance(window, 'day')
    viirs_info = (ee_utils.eeDate_to_string(window_start),ee_utils.eeDate_to_string(window_end)) 

    image_collections, time_sampling = _get_all_image_collections(viirs_info = viirs_info)

    drought = image_collections['drought'].filterDate(
      window_start.advance(-lag - time_sampling['drought'], 'day'),
      window_start.advance(
          -lag, 'day')).median().reproject(projection.atScale(10000)).resample('bicubic')
    
    vegetation = image_collections['vegetation'].filterDate(
      window_start.advance(-lag - time_sampling['vegetation'], 'day'),
      window_start.advance(
          -lag, 'day')).median().reproject(projection).resample('bicubic')
    
    weather = image_collections['weather'].filterDate(
      window_start.advance(-lag - time_sampling['weather'], 'day'),
      window_start.advance(-lag, 'day')).median().reproject(
          projection.atScale(10000)).resample('bicubic')
    
    modis_prev_fire = image_collections['modis_fire'].filterDate(
      window_start.advance(-lag - time_sampling['modis_fire'], 'day'),
      window_start.advance(-lag, 'day')).map(
          ee_utils.remove_mask).max().rename('modis_PrevFireMask')

    viirs_prev_fire = image_collections['viirs_fire'].filterDate(
      window_start.advance(-lag - time_sampling['viirs_fire'], 'day'),
      window_start.advance(-lag, 'day')).map(
          ee_utils.remove_mask).max().rename('viirs_PrevFireMask')
    
    modis_fire = image_collections['modis_fire'].filterDate(window_start, window_end).map(
      ee_utils.remove_mask).max().rename('modis_FireMask')

    viirs_fire = image_collections['viirs_fire'].filterDate(window_start,window_end).map(
      ee_utils.remove_mask).max().rename('viirs_FireMask')
    
    detection = viirs_prev_fire.rename('detection') # can use the MODIS data for detection, but have to subtract 6 from all measurements
    
    return [vegetation, drought, weather, modis_prev_fire, viirs_prev_fire, modis_fire, viirs_fire, detection]

### export dataset code

In [7]:
def _export_dataset(
    bucket,
    folder,
    prefix,
    start_date,
    start_days,
    geometry,
    kernel_size,
    sampling_scale,
    num_samples_per_file,
    verbose = True,
):
    """Exports the dataset TFRecord files for wildfire risk assessment.
    
    Args:
    bucket: Google Cloud bucket
    folder: Folder to which to export the TFRecords.
    prefix: Export file name prefix.
    start_date: Start date for the EE data to export.
    start_days: Start day of each time chunk to export.
    geometry: EE geometry from which to export the data.
    kernel_size: Size of the exported tiles (square).
    sampling_scale: Resolution at which to export the data (in meters).
    num_samples_per_file: Approximate number of samples to save per TFRecord
      file.
    """
    
    def _verify_and_export_feature_collection(
      num_samples_per_export,
      feature_collection,
      file_count,
      features,
    ):
        """Wraps the verification and export of the feature collection.
        
        Verifies the size of the feature collection and triggers the export when
        it is larger than `num_samples_per_export`. Resets the feature collection
        and increments the file count at each export.
        
        Args:
          num_samples_per_export: Approximate number of samples per export.
          feature_collection: The EE feature collection to export.
          file_count: The TFRecord file count for naming the files.
          features: Names of the features to export.
        
        Returns:
          `(feature_collection, file_count)` tuple of the current feature collection
            and file count.
        """
        feature_collection, size_count = _verify_feature_collection(
            feature_collection)
        if size_count > num_samples_per_export:
            # print("EXPORTING A FEATURE COLLECTION OF SIZE",size_count) # debugging
            ee_utils.export_feature_collection(
              feature_collection,
              description=prefix + '_{:03d}'.format(file_count),
              bucket=bucket,
              folder=folder,
              bands=features,
            )
            file_count += 1
            feature_collection = ee.FeatureCollection([])
        return feature_collection, file_count

    # Get all feature names (inputs and responses)
    features = _get_all_feature_bands() + _get_all_response_bands()
    if verbose:
        print("Features to be extracted:")
        print(features)

    # Next, obtain the data which are not time-sliced, or not time-dependent
    
    elevation = ee_utils.get_image(ee_utils.DataType.ELEVATION_SRTM) # Elevation: only a single image
    
    chili = ee_utils.get_image(ee_utils.DataType.CHILI).rename('chili') # CHILI: only a single image
    
    impervious_coll = ee_utils.get_image_collection(ee_utils.DataType.IMPERVIOUS) # impervious: use 2016 data
    impervious = impervious_coll.filterDate('2016-01-01', '2017-01-01').first()
    
    water_cover = ee_utils.get_image(ee_utils.DataType.WTR_COVER).rename('water') # water cover: single image
    water_cover = water_cover.unmask() # unmasking is crucial for this dataset, or extract samples will return empty
    
    # population density
    population_coll = ee_utils.get_image_collection(ee_utils.DataType.POPULATION)
    population = population_coll.filterDate('2015-01-01',
                                     '2018-01-01').first().rename('population')

    # fuel types
    fuel = ee_utils.get_image(ee_utils.DataType.LATENT_FUELS)
    fuel1 = fuel.select('b1').rename('fuel1')
    fuel2 = fuel.select('b2').rename('fuel2')
    fuel3 = fuel.select('b3').rename('fuel3')

    
    all_days = []
    for day in start_days:
        for i in range(7):
            all_days.append(day + i)
    
    window = 1
    sampling_limit_per_call = 60
    
    
    file_count = 0
    total_feature_collection = ee.FeatureCollection([])
    
    for start_day in all_days:
        window_start = start_date.advance(start_day, 'days')
        if verbose:
            print("Looking at date",window_start.format().getInfo())

        # aggregate the features as a list of images, with collections time-sliced
        time_slices = _get_time_slices(window_start)
        image_list = [elevation, chili, impervious,water_cover,population,fuel1,fuel2,fuel3] + time_slices[:-1]
        detection = time_slices[-1]
        
        arrays = ee_utils.convert_features_to_arrays(image_list, kernel_size)
        to_sample = detection.addBands(arrays)


        if verbose: print("Getting detection count....")
        fire_count = ee_utils.get_detection_count(
            detection,
            geometry=geometry,
            sampling_scale= 10 * sampling_scale, # sampling scale is 10km = 20*500m here, which defines the rule to discriminate 
                                                #        spatially separated/distinct fires
        )
        
        if fire_count > 0:
            if verbose: print("Found a fire: fire_count =",fire_count)
                
            samples = ee_utils.extract_samples(
              to_sample,
              fire_count,
              geometry, 
              sampling_limit_per_call=sampling_limit_per_call,
              resolution=sampling_scale,
              date = window_start,
            )
            
            total_feature_collection = total_feature_collection.merge(samples)

            total_feature_collection, file_count = _verify_and_export_feature_collection(
              num_samples_per_file, total_feature_collection, file_count, features)
            
    # Export the remaining feature collection
    _verify_and_export_feature_collection(0, total_feature_collection, file_count,
                                    features)

In [12]:
def export_ml_datasets(
    bucket,
    folder,
    start_date,
    end_date,
    prefix = '',
    kernel_size = 64,
    sampling_scale = 500,
    eval_split_ratio = 0.125,
    num_samples_per_file = 1000,
    coordinates = ee_utils.COORDINATES['OREGON']
    ):
    """Exports the ML dataset TFRecord files for wildfire risk assessment.
    
    Export is to Google Cloud Storage.
    
    Args:
    bucket: Google Cloud bucket
    folder: Folder to which to export the TFRecords.
    start_date: Start date for the EE data to export.
    end_date: End date for the EE data to export.
    prefix: File name prefix to use.
    kernel_size: Size of the exported tiles (square).
    sampling_scale: Resolution at which to export the data (in meters).
    eval_split_ratio: Split ratio for the divide between training and evaluation
      datasets.
    num_samples_per_file: Approximate number of samples to save per TFRecord
      file.
    """
    
    split_days = ee_utils.split_days_into_train_eval_test(
      start_date, end_date, split_ratio=eval_split_ratio, window_length_days=8)
    
    for mode in ['train', 'eval', 'test']:
        print("Beginning export process in mode",mode)
        sub_prefix = f'{mode}_{prefix}'
        _export_dataset(
            bucket=bucket,
            folder=folder,
            prefix=sub_prefix,
            start_date=start_date,
            start_days=split_days[mode],
            geometry=ee.Geometry.Rectangle(coordinates),
            kernel_size=kernel_size,
            sampling_scale=sampling_scale,
            num_samples_per_file=num_samples_per_file)

### Define parameters and begin data export

In [9]:
config = {
    'bucket':'ndws_viirs_dataset',
    'folder':'conus_west',
    'start_date':ee.Date('2018-01-01'),
    'end_date':ee.Date('2023-12-31'),
    'prefix':'conus_west_ndws',
    'kernel_size':64, # size of export images in pixels
    'sampling_scale':500, # size of pixel in meters
    'eval_split_ratio':0.2,
    'num_samples_per_file':100,
    'coordinates':ee_utils.COORDINATES['US_WEST']
}

In [18]:
with open('ndws_conus_west.json','w') as json_file:
    json.dump({x:str(config[x]) for x in config},json_file)

In [10]:
res = export_ml_datasets(
      bucket=config['bucket'],
      folder=config['folder'],
      start_date=config['start_date'],
      end_date=config['end_date'],
      prefix=config['prefix'],
      kernel_size=config['kernel_size'],
      sampling_scale=config['sampling_scale'],
      eval_split_ratio=config['eval_split_ratio'],
      num_samples_per_file=config['num_samples_per_file'],
      coordinates=config['coordinates'],
  )

Beginning export process in mode train
Features to be extracted:
['elevation', 'chili', 'impervious', 'water', 'population', 'fuel1', 'fuel2', 'fuel3', 'NDVI', 'pdsi', 'pr', 'sph', 'th', 'tmmn', 'tmmx', 'vs', 'erc', 'bi', 'modis_PrevFireMask', 'viirs_PrevFireMask', 'modis_FireMask', 'viirs_FireMask']
Looking at date 2018-02-18T00:00:00
Reading VIIRS dataframes...
Concatenating dataframes...
Creating Earth Engine images...


  0%|          | 0/2 [00:00<?, ?it/s]

Getting detection count....
Looking at date 2018-02-19T00:00:00
Reading VIIRS dataframes...
Concatenating dataframes...
Creating Earth Engine images...


  0%|          | 0/2 [00:00<?, ?it/s]

Getting detection count....
Looking at date 2018-02-20T00:00:00
Reading VIIRS dataframes...
Concatenating dataframes...
Creating Earth Engine images...


...

Getting detection count....
Looking at date 2018-08-02T00:00:00
Reading VIIRS dataframes...
Concatenating dataframes...
Creating Earth Engine images...


  0%|          | 0/2 [00:00<?, ?it/s]

Getting detection count....
Looking at date 2018-08-03T00:00:00
Reading VIIRS dataframes...
Concatenating dataframes...
Creating Earth Engine images...


  0%|          | 0/2 [00:00<?, ?it/s]

Getting detection count....
Verified a feature collection of size 90
EXPORTING A FEATURE COLLECTION OF SIZE 90
